In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

*Load necessary libraries*

In [10]:
data_x = np.linspace(1, 8, 100)[:, np.newaxis]
data_y = np.polyval([1, -14, 59, -70], data_x) \
        + 1.5 * np.sin(data_x) + np.random.randn(100, 1)

*Generate our data*

In [3]:
model_order = 5
data_x = np.power(data_x, range(model_order))
data_x /= np.max(data_x, axis=0)

*Add intercept data and normalize*

In [4]:
order = np.random.permutation(len(data_x))
portion = 20
test_x = data_x[order[:portion]]
test_y = data_y[order[:portion]]
train_x = data_x[order[portion:]]
train_y = data_y[order[portion:]]

*Shuffle data and produce train and test sets*

In [5]:
init_param = lambda shape: tf.zeros(shape, dtype=tf.float32)

with tf.name_scope("IO"):
    inputs = tf.placeholder(tf.float32, [None, model_order], name="X")
    outputs = tf.placeholder(tf.float32, [None, 1], name="Yhat")

with tf.name_scope("LR"):
    W = tf.Variable(init_param([model_order, 1]), name="W")
    y = tf.matmul(inputs, W)
    
with tf.name_scope("train"):
    learning_rate = tf.Variable(0.5, trainable=False)
    cost_op = tf.reduce_mean(tf.pow(y-outputs, 2))
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_op)

*Create TensorFlow graph*

In [6]:
tolerance = 1e-3

# Perform Stochastic Gradient Descent
epochs = 1
last_cost = 0
alpha = 0.4
max_epochs = 50000

sess = tf.Session()
print "Beginning Training"
with sess.as_default():
    init = tf.initialize_all_variables()
    sess.run(init)
    sess.run(tf.assign(learning_rate, alpha))
    writer = tf.train.SummaryWriter("/tmp/tboard", sess.graph) # Create TensorBoard files
    while True:

        sess.run(train_op, feed_dict={inputs: train_x, outputs: train_y})
            
        # Keep track of our performance
        if epochs%100==0:
            cost = sess.run(cost_op, feed_dict={inputs: train_x, outputs: train_y})
            print "Epoch: %d - Error: %.4f" %(epochs, cost)

            # Stopping Condition
            if abs(last_cost - cost) < tolerance or epochs > max_epochs:
                print "Converged."
                break
            last_cost = cost
            
        epochs += 1
    
    w = W.eval()
    print "w =", w
    print "Test Cost =", sess.run(cost_op, feed_dict={inputs: test_x, outputs: test_y})


Beginning Training
Epoch: 100 - Error: 46.5217
Epoch: 200 - Error: 45.4410
Epoch: 300 - Error: 44.4665
Epoch: 400 - Error: 43.5364
Epoch: 500 - Error: 42.6334
Epoch: 600 - Error: 41.7520
Epoch: 700 - Error: 40.8904
Epoch: 800 - Error: 40.0480
Epoch: 900 - Error: 39.2242
Epoch: 1000 - Error: 38.4185
Epoch: 1100 - Error: 37.6305
Epoch: 1200 - Error: 36.8598
Epoch: 1300 - Error: 36.1061
Epoch: 1400 - Error: 35.3690
Epoch: 1500 - Error: 34.6481
Epoch: 1600 - Error: 33.9430
Epoch: 1700 - Error: 33.2535
Epoch: 1800 - Error: 32.5791
Epoch: 1900 - Error: 31.9195
Epoch: 2000 - Error: 31.2745
Epoch: 2100 - Error: 30.6436
Epoch: 2200 - Error: 30.0266
Epoch: 2300 - Error: 29.4232
Epoch: 2400 - Error: 28.8331
Epoch: 2500 - Error: 28.2559
Epoch: 2600 - Error: 27.6915
Epoch: 2700 - Error: 27.1394
Epoch: 2800 - Error: 26.5995
Epoch: 2900 - Error: 26.0715
Epoch: 3000 - Error: 25.5550
Epoch: 3100 - Error: 25.0500
Epoch: 3200 - Error: 24.5560
Epoch: 3300 - Error: 24.0729
Epoch: 3400 - Error: 23.6004
Epoc

*Perform gradient descent to learn model*

In [7]:
y_model = np.polyval(w[::-1], np.linspace(0,1,200))
plt.plot(np.linspace(0,1,200), y_model, c='g', label='Model')
plt.scatter(train_x[:,1], train_y, c='b', label='Train Set')
plt.scatter(test_x[:,1], test_y, c='r', label='Test Set')
plt.grid()
plt.legend(loc='upper left')
plt.xlabel('X')
plt.ylabel('Y')
plt.xlim(0,1)
plt.show()

*Plot the model obtained*